In [ ]:
!pip install bigframes --quiet
!pip install SQLAlchemy --quiet
!pip install sqlalchemy-bigquery --quiet

import IPython
print( 'restarting kernel...' )
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.7/293.7 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.49 which is incompatible.
restarting kernel...


{'status': 'ok', 'restart': True}

In [ ]:
# @title Specify Project details and LOCATION of the BQ table

PROJECT_ID = ""  # @param {type:"string"}
DB_PROJECT_ID = "bigquery-public-data"  # @param {type:"string"}

#DB_PROJECT_ID = "cloud-llm-preview1"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
#DATASET_ID = 'blackbelt_capstone_healthcare' # @param {type:"string"}
DATASET_ID = 'ml_datasets' # @param {type:"string"}

#@markdown ### Enter the topic name and subscription to be used for pub/sub
TOPIC_NAME="customer-chat" # @param {type:"string"}
SUBSCRIPTION="colab-sub" # @param {type:"string"}
OUTPUT_TABLE_NAME="penguins_model" # @param {type:"string"}


import sys
IN_COLAB = 'google.colab' in sys.modules
%env IN_COLAB=$IN_COLAB

!gcloud config set project $PROJECT_ID -q
!gcloud config get project

env: IN_COLAB=True
Updated property [core/project].
kallogjeri-project-345114


In [ ]:
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
from google.cloud import bigquery

def create_ds_view( tbl_name ):
  view_name = f"vw_{tbl_name}"
  view_id = f"{PROJECT_ID}.{DATASET_ID}.{view_name}"
  source_id = f"{DB_PROJECT_ID}.{DATASET_ID}.{tbl_name}"
  view = bigquery.Table(view_id)

  view.view_query = f"SELECT * FROM `{source_id}`"

  # Make an API request to create the view.
  view = bq_client.create_table(view, exists_ok=True)
  #print(f"Created {view.table_type}: {str(view.reference)}")
  fq_view_name = f"{DATASET_ID}.{view_name}"
  return view_name, fq_view_name



query = f"""SELECT table_name \
  FROM `{DB_PROJECT_ID}.{DATASET_ID}`.INFORMATION_SCHEMA.COLUMNS """


# Create a BigQuery client.
bq_client = bigquery.Client(project=PROJECT_ID)

table_uri = f"bigquery://{PROJECT_ID}/{DATASET_ID}"
engine = create_engine(
    f"bigquery://{DB_PROJECT_ID}/{DATASET_ID}?user_supplied_client=True",
    connect_args={'client': bq_client}
)

tbl_names = set(engine.execute(query).unique().fetchall())
from itertools import chain
tbl_names = list(chain(*tbl_names))

# Create the dataset if it doesn't exist
try:
    bq_client.get_dataset(DATASET_ID)
except:
    # The dataset doesn't exist, so create it.
    dataset = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
    bq_client.create_dataset(dataset)


view_struct = list(map( create_ds_view, tbl_names ))

view_names = [view[0] for view in view_struct]
print(view_names)
fq_view_names = [view[1] for view in view_struct]


table_str = "','".join(tbl_names)
view_str = "','".join(view_names)

column_query =  f"SELECT table_name, column_name \
  FROM `{PROJECT_ID}.{DATASET_ID}`.INFORMATION_SCHEMA.COLUMNS \
  WHERE table_name in ('{view_str}')"

print(column_query)

columns =  list(engine.execute(column_query).unique().fetchall())
#print(columns[1])
columns = list(chain(*columns))
#print(columns)

column_names = list(map(lambda x, y: f"{x}.{y}", columns[::2], columns[1::2]))

<ipython-input-2-4d943eb82a33>:35: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  tbl_names = set(engine.execute(query).unique().fetchall())


RefreshError: ignored

In [ ]:
# Start a BigFrames session
import bigframes

ops = bigframes.BigQueryOptions()
ops.project = PROJECT_ID
ops.dataset = "ml_datasets"
ops.table = "vw_penguins"

# Connect to a BigQuery session
session = bigframes.connect(context=ops)

# Initialize a dataframe for a BigQuery table
df = session.read_gbq("bigquery-public-data.ml_datasets.penguins")

# View the DataFrame
df

# View the column names in the dataframe (aka columns names in the table)
df.columns

# View the table schema
df.dtypes
# Select a subset of columns
df = df[[
    "species",
    "island",
    "body_mass_g",
]]
df

# View the first ten values of a series
df['body_mass_g'].head(10)

# Compute the mean of a series
df['body_mass_g'].mean()

# Filter the dataframe
df[df['body_mass_g'] >= 4000.0]

#####################################
#       Remote Functions            #
#####################################

# BigFrames gives you the ability to turn your custom scalar functions into a BigQuery remote function. It requires the GCP project to be set up appropriately and the user having sufficient privileges to use them. One can find more details on it via help command.
#help(bigframes.remote_function)

# Run the custom function on the BigFrames dataframe
@session.remote_function([float], str, bigquery_connection='bigframes-rf-conn')
def get_bucket(num):
    if not num: return "NA"
    boundary = 4000
    return "at_or_above_4000" if num >= boundary else "below_4000"

#####################################
#       ML API                      #
#####################################

# Start a session and initialize a dataframe for a BigQuery table
df = session.read_gbq("bigquery-public-data.ml_datasets.penguins")
df

# Clean and prepare the data
# filter down to the data we want to analyze
adelie_data = df[df.species == "Adelie Penguin (Pygoscelis adeliae)"]

# drop the columns we don't care about
adelie_data = adelie_data.drop(columns=["species"])

# drop rows with nulls to get our training data
training_data = adelie_data.dropna()

# take a peek at the training data
training_data



/usr/local/lib/python3.10/dist-packages/bigframes/session.py:1158: UserWarning: No explicit location is set, so using location US for the session.
  return Session(context)


HTML(value='Query job 8597d4a0-1234-4ec1-9e72-7f0808a7767e is RUNNING. <a target="_blank" href="https://consol…

HTML(value='Query job 2aef9e8a-beba-47a8-af65-80faf7b85c28 is DONE. 2.7 kB processed. <a target="_blank" href=…

HTML(value='Query job 09d2b0c1-4bfe-4f53-b45d-ce457ca0f8b9 is RUNNING. <a target="_blank" href="https://consol…

HTML(value='Query job b670d9e7-fdaf-4761-95c2-2b0e04589fde is DONE. 28.9 kB processed. <a target="_blank" href…

HTML(value='Query job dcfb2524-05f4-492f-ba50-6a76916003d4 is DONE. 31.7 kB processed. <a target="_blank" href…

HTML(value='Query job 252f9c88-267b-4fd2-8078-c31efaaa1842 is DONE. 0 Bytes processed. <a target="_blank" href…

,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Dream,36.6,18.4,184.0,3475.0,FEMALE
1,Dream,39.8,19.1,184.0,4650.0,MALE
2,Dream,40.9,18.9,184.0,3900.0,MALE
4,Dream,37.3,16.8,192.0,3000.0,FEMALE
5,Dream,43.2,18.5,192.0,4100.0,MALE
9,Dream,40.2,20.1,200.0,3975.0,MALE
10,Dream,40.8,18.9,208.0,4300.0,MALE
11,Dream,39.0,18.7,185.0,3650.0,MALE
12,Dream,37.0,16.9,185.0,3000.0,FEMALE
14,Dream,34.0,17.1,185.0,3400.0,FEMALE


In [ ]:
# pick feature columns and label column
#feature_columns = training_data[['island', 'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'sex']]
feature_columns = training_data[['island', 'culmen_length_mm', 'flipper_length_mm', 'sex']]
feature_columns.head()

label_columns = training_data[['body_mass_g']]

# also get the rows that we want to make predictions for (i.e. where the feature column is null)
missing_body_mass = adelie_data[adelie_data.body_mass_g.isnull()]

# Train and evaluate a linear regression model using the ML API
from bigframes.ml.linear_model import LinearRegression

# as in scikit-learn, a newly created model is just a bundle of parameters
# default parameters are fine here
model = LinearRegression()

# this will train a temporary model in BigQuery Machine Learning
model.fit(feature_columns, label_columns)

# check how the model performed, using the automatic test/training data split chosen by BQML
model.score(feature_columns, label_columns)

# Make predictions using the model
model.predict(missing_body_mass)

# Create the dataset if it doesn't exist
try:
    bq_client.get_dataset(DATASET_ID)
except:
    # The dataset doesn't exist, so create it.
    dataset = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
    bq_client.create_dataset(dataset)


# Save the trained model to BigQuery, so we can load it later
model.to_gbq(f"{DATASET_ID}.{OUTPUT_TABLE_NAME}", replace=True)

HTML(value='Query job 7a5628b0-a8c4-4231-b37b-87b865941020 is RUNNING. <a target="_blank" href="https://consol…

HTML(value='Query job ce697f5c-8c5c-4273-b03f-0faef156b80b is DONE. 31.9 kB processed. <a target="_blank" href…

HTML(value='Query job ce7da4f5-a845-49f8-86fa-b42798b8788a is RUNNING. <a target="_blank" href="https://consol…

HTML(value='Query job 9d320545-8bef-4ba5-bb81-ccd94b99af18 is RUNNING. <a target="_blank" href="https://consol…

HTML(value='Query job 475d6f6b-c7f4-494d-a143-991b3ae72b39 is DONE. 8 Bytes processed. <a target="_blank" href…

HTML(value='Copy job 9df251ad-f728-4d48-b1b2-80e2210d0f8c is RUNNING. <a target="_blank" href="https://console…

LinearRegression()